In [1]:
import os
import traceback
import logging
import os.path
import networkx as nx
import pandas as pd
import numpy as np
import pylab as pl
from matplotlib import collections  as mc
import glob
import math

In [2]:
# simversion = '_v01'
simversion = '_plasmid_v01'
simversionPangraph = '_assemBiGraph2_v01' #'_assemGraph2_v01': use assembly graph v2; _assemBiGraph2_v01: if (u,v) in G, so is (v-, u-)
# simversion = '_plasmid_v02'
# simversion = '_plasmid_v03'
# simversion = '_plasmid_v04'
# 1 True, 0 False
prepare_database_4_multicsar = 0
run_art = 0
run_spades = 0
run_panta = 0
split_paralogs = 1
run_multicsar = 0
run_ragout = 0
run_pangraph = 1
# pangenome_data = '/data/hoan/amromics/data/ncbi/Kp100random/' # for pangraph, Ragout, multi-CSAR #v3
pangenome_data = '/data/hoan/amromics/data/ncbi/Ecoli50random/'
# pangenome_data = '/data/hoan/amromics/data/ncbi/Kptest/' # for pangraph, Ragout, multi-CSAR
scaffold_out_dir = '/data/hoan/amromics/genome-graph/scaffold_output/'

In [3]:
if prepare_database_4_multicsar==1:
    os.system('rm -r ' + pangenome_data + 'fna/')
    os.system('mkdir ' + pangenome_data + 'fna/')
    os.system('cp -r '+ pangenome_data + '*.fna.gz '+ pangenome_data + 'fna/')
    os.system('~/miniconda3/envs/panta/bin/gzip -d ' + pangenome_data + 'fna/*.fna.gz')

### Simulate reads using ART

In [4]:
# https://github.com/scchess/Art/blob/master/art_illumina_README

In [5]:
sim_dir = '/data/hoan/amromics/simulation/'
art_bin = '/data/hoan/amromics/simulation/art_bin/./art_illumina'

In [142]:
# ref_data = '/data/hoan/amromics/simulation/references/GCF_000240185.1_ASM24018v2_genomic.fasta'
# ref_data = '/data/hoan/amromics/simulation/references/GCF_000240185.1_ASM24018v2_genomic.fna'
# ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna'
# ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_002899495.1_ASM289949v1_genomic.fna'  ## Mau nay co van de.
# ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_004135915.1_ASM413591v1_genomic.fna'
# ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_006351885.1_ASM635188v1_genomic.fna'
ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_014169015.1_ASM1416901v1_genomic.fna'
# ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_900635215.1_39859_D02_genomic.fna'
sim_name = ref_data.split("/")[-2] + '_' + ref_data.split("/")[-1][:-1][:13]
sim_output = '/data/hoan/amromics/simulation/art_output/sim_reads_' + sim_name + simversion[-4:]
# sim_output = '/data/hoan/amromics/simulation/art_output/paired_dat_test'

In [143]:
sim_output

'/data/hoan/amromics/simulation/art_output/sim_reads_Ecolitest_GCF_014169015_v01'

In [144]:
if run_art: 
    if simversion[-3:]=='v01':
        os.system(art_bin+' -ss HS20 -sam -i '+ref_data+' -p -l 100 -f 70 -m 400 -s 60 -o '+sim_output) # _v01
    if simversion[-3:]=='v02':
        os.system(art_bin+' -ss HS20 -sam -i '+ref_data+' -p -l 100 -f 70 -m 400 -s 10 -o '+sim_output) # _v02
    if simversion[-3:]=='v03':
        os.system(art_bin+' -ss MSv3 -sam -i '+ref_data+' -p -l 250 -f 70 -m 400 -s 60 -o '+sim_output) # _v03
    if simversion[-3:]=='v04':
        os.system(art_bin+' -ss MSv3 -sam -i '+ref_data+' -p -l 250 -f 70 -m 400 -s 10 -o '+sim_output) # _v04

### Run SPADES assembly

In [145]:
spades_output = '/data/hoan/amromics/simulation/art_output/' + 'spades_output_' + sim_name + simversion[-4:]
spades_output

'/data/hoan/amromics/simulation/art_output/spades_output_Ecolitest_GCF_014169015_v01'

In [146]:
if run_spades:
    spades_bin ='~/miniconda3/envs/amromics/bin/spades.py'
    os.system(spades_bin+' --isolate -1 '+sim_output+'1.fq -2 '+sim_output+'2.fq -t 50 -o '+spades_output)

### Run panta

In [147]:
simversionRef = '_'+ pangenome_data.split('/')[-2] +'_'+ sim_name + simversion
simversionRef

'_Ecoli50random_Ecolitest_GCF_014169015_plasmid_v01'

In [148]:
# move spades ouput to panta dir
print('Pangenome data: ', pangenome_data)
if run_panta:
    os.system('cp '+spades_output+'/contigs.fasta ' + pangenome_data + 'g1.fna')
    os.system('rm ' + pangenome_data + 'g1.fna.gz')
    os.system('~/miniconda3/envs/panta/bin/gzip -k ' + pangenome_data + 'g1.fna')

Pangenome data:  /data/hoan/amromics/data/ncbi/Ecoli50random/


In [149]:
# Run panta (change output dir)
# https://askubuntu.com/questions/1252439/not-able-to-activate-conda-environment-through-os-system-command-in-python
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate panta && '
panta_bin =conda_dir+'python /data/hoan/amromics/panta/panta.py'
panta_output = '/data/hoan/amromics/panta/examples/test/pantaOut' + simversionRef
# panta_output = '/data/hoan/amromics/panta/examples/test/output_Kp100' 
if split_paralogs == 0:
    panta_output = panta_output + '_nosplit'
print('PANTA OUTPUT: ', panta_output)
if run_panta:
    if split_paralogs:
        cmd_panta = panta_bin + ' -p init -a ' +pangenome_data+ '*.fna.gz -o '+panta_output +' -as -s -i 85 -c 20 -e 0.01'
    else:
        print("No split ortholog")
        cmd_panta = panta_bin + ' -p init -a ' +pangenome_data+ '*.fna.gz -o '+panta_output +' -as'
    os.system(cmd_panta)

PANTA OUTPUT:  /data/hoan/amromics/panta/examples/test/pantaOut_Ecoli50random_Ecolitest_GCF_014169015_plasmid_v01


### Run pangraph

In [150]:
try:
    %load_ext autoreload
    %autoreload 2
except Exception as e:
    logging.error(traceback.format_exc())
from pangraph import PanGraph

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [151]:
# set parameters
data_dir = panta_output 
incomplete_sample_name = 'g1'
assem_dir = spades_output
fasta_gen = 'partial' # 'all', 'partial'

In [152]:
simversionRefNew = '_'+ pangenome_data.split('/')[-2] +'_'+ sim_name + simversionPangraph
# pangraph_output_greedy_default = scaffold_out_dir + 'pangraph_greedy'+ simversionRef + '.fasta'
# pangraph_output_opt_default = scaffold_out_dir + 'pangraph_opt'+ simversionRef + '.fasta'
pangraph_output_greedy = scaffold_out_dir + 'pangraph_greedy'+ simversionRefNew + '.fasta'
pangraph_output_opt = scaffold_out_dir + 'pangraph_opt'+ simversionRefNew + '.fasta'

In [153]:
pangraph_output_opt

'/data/hoan/amromics/genome-graph/scaffold_output/pangraph_opt_Ecoli50random_Ecolitest_GCF_014169015_assemBiGraph2_v01.fasta'

In [154]:
if run_pangraph:
    # https://stackoverflow.com/questions/4990718/how-can-i-write-a-try-except-block-that-catches-all-exceptions
    pangraph = PanGraph(sample_info=None, gene_info=None, gene_position=None)
    try:
        maximum_matching = 'greedy'
        pangraph.run_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_greedy, maximum_matching)
    except Exception as e:
        logging.error(traceback.format_exc())
        # Logs the error appropriately. 

Should we use target sequence as reference, hence, the strand will be ok
Set minimum on number of nucleotides =  10 NUMBER OF COMPUTED CONTIGS: 234
Clip the matrix 0.2 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Use modified  assembly graph
Use union graph: overlap + assembly
Please set min_weight back to 1.0
Will test this, Ok?
Will test this, Ok?
Success! File written


In [155]:
if run_pangraph:
    try:
        maximum_matching = 'opt'
        pangraph.RERUN_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_opt, maximum_matching)
    except Exception as e:
        logging.error(traceback.format_exc())
        # Logs the error appropriately. 

Compute maximum matching
Will test this, Ok?
Will test this, Ok?
Success! File written


In [156]:
# try:
#     maximum_matching = 'greedy'
#     pangraph_output_greedy = spades_output + '/contigs_concat_'+ maximum_matching+ simversionRef + '.fasta'
#     pangraph.RERUN_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_opt, maximum_matching)
# except Exception as e:
#     logging.error(traceback.format_exc())
#     # Logs the error appropriately. 

### Run scaffold methods

In [157]:
### Multi-CSAR
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && '
csar_bin = conda_dir + '/data/hoan/amromics/assembly/Multi-CSAR/./multi-csar.php'

In [158]:
multicsar_output = scaffold_out_dir + 'multicsar/multicsar' + simversionRef
if run_multicsar:
    os.system('rm ' + pangenome_data + 'fna/g1*')
    os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'fna/ --nuc -o '+multicsar_output)
    # os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'*.fna --nuc -o '+multicsar_output)

In [159]:
multicsar_output

'/data/hoan/amromics/genome-graph/scaffold_output/multicsar/multicsar_Ecoli50random_Ecolitest_GCF_014169015_plasmid_v01'

In [160]:
### Ragout
ragout_bin = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && ragout '
ragout_output = scaffold_out_dir + 'multicsar/ragout' + simversionRef
if run_ragout:
    # write receipt file
    ref_files_list = glob.glob(pangenome_data+'fna/*')
    n_files = len(ref_files_list)
    receipt_file_dir = '/data/hoan/amromics/assembly/Ragout/kp100'+simversionRef+'.rcp'
    f = open(receipt_file_dir, "w")
    f.write('.references = ')
    for idx in range(n_files-1):
        f.write('r' + str(idx) + ',')
    f.write('r'+str(n_files-1)+'\n')
    f.write(".target = mg1655\n\n")
    for idx in range(n_files):
        f.write('r'+str(idx)+'.fasta =' + ref_files_list[idx] + '\n')
    f.write('mg1655.fasta = ' + spades_output+'/contigs.fasta\n')
    f.close()

In [161]:
if run_ragout:
    os.system(ragout_bin + receipt_file_dir + ' --outdir ' + ragout_output + ' --refine')
    # ragout kp100_v2.rcp --outdir output_Kp100p_v2/ --refine

In [162]:
print(ragout_output)

/data/hoan/amromics/genome-graph/scaffold_output/multicsar/ragout_Ecoli50random_Ecolitest_GCF_014169015_plasmid_v01


### Quast 

In [163]:
quast_output = '/data/hoan/amromics/genome-graph/scaffold_output/quastResults' + simversionRefNew
quast_output

'/data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecoli50random_Ecolitest_GCF_014169015_assemBiGraph2_v01'

In [164]:
quast_bin = 'python /data/hoan/amromics/spades_quast/quast-5.2.0/quast.py '
spades_output_fasta = spades_output+'/contigs.fasta'
multicsar_output_fasta = multicsar_output +'/multi-csar.nuc.out.fna'
ragout_output_fasta = ragout_output + '/mg1655_scaffolds.fasta'
if 1:
    if os.path.isfile(ragout_output_fasta):
        os.system(quast_bin + pangraph_output_opt+' '+ pangraph_output_greedy+' '+ multicsar_output_fasta+' '+ragout_output_fasta+' '+spades_output_fasta+' '+
        '-l "Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, Ragout, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')
    else:
        os.system(quast_bin + pangraph_output_opt+' '+ pangraph_output_greedy+' '+ multicsar_output_fasta+' '+spades_output_fasta+' '+
        '-l "Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')

/data/hoan/amromics/spades_quast/quast-5.2.0/quast.py /data/hoan/amromics/genome-graph/scaffold_output/pangraph_opt_Ecoli50random_Ecolitest_GCF_014169015_assemBiGraph2_v01.fasta /data/hoan/amromics/genome-graph/scaffold_output/pangraph_greedy_Ecoli50random_Ecolitest_GCF_014169015_assemBiGraph2_v01.fasta /data/hoan/amromics/genome-graph/scaffold_output/multicsar/multicsar_Ecoli50random_Ecolitest_GCF_014169015_plasmid_v01/multi-csar.nuc.out.fna /data/hoan/amromics/simulation/art_output/spades_output_Ecolitest_GCF_014169015_v01/contigs.fasta -l Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, SPADES -r /data/hoan/amromics/data/ncbi/Ecolitest/GCF_014169015.1_ASM1416901v1_genomic.fna -o /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecoli50random_Ecolitest_GCF_014169015_assemBiGraph2_v01 --silent --extensive-mis-size 5000 --local-mis-size 3000


System information:
  OS: Linux-3.10.0-1160.15.2.el7.x86_64-x86_64-with-redhat-7.9-Nitrogen (linux_64)
  Python version: 2.7.5
  CPUs numb

Unable to parse the pattern
Unable to parse the pattern
Unable to parse the pattern


Done.

2023-01-28 16:18:23
Running Contig analyzer...
Done.

2023-01-28 16:18:25
Running NA-NGA calculation...
Done.

2023-01-28 16:18:26
Running Genome analyzer...
  NOTICE: No file with genomic features were provided. Use the --features option if you want to specify it.

  NOTICE: No file with operons were provided. Use the -O option if you want to specify it.
Done.

NOTICE: Genes are not predicted by default. Use --gene-finding or --glimmer option to enable it.

2023-01-28 16:18:28
Creating large visual summaries...
This may take a while: press Ctrl-C to skip this step..
  1 of 2: Creating PDF with all tables and plots...
  2 of 2: Creating Icarus viewers...
Done

2023-01-28 16:18:30
RESULTS:
  Text versions of total report are saved to /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecoli50random_Ecolitest_GCF_014169015_assemBiGraph2_v01/report.txt, report.tsv, and report.tex
  Text versions of transposed total report are saved to /data/hoan/amromics/genome-graph/scaf

In [165]:
# stop

In [166]:
# pangraph.edge_df0[-20:]

In [167]:
# # pangraph.H["weight"][0:4]
# iteration = 0
# for u,v,a in pangraph.H.edges(data=True):
#     print(a)
#     iteration += 1
#     if iteration > 10:
#         break;

In [168]:
# nx.write_gml(pangraph.assembly_graph, quast_output+'/assembly_graph.gml')

In [169]:
pangraph.assembly_graph.number_of_edges()

3240

In [170]:
len(list(nx.weakly_connected_components(pangraph.assembly_graph)))

1

In [171]:
# pangraph.weighted_CG.nodes()

In [172]:
len(list(nx.weakly_connected_components(pangraph.H)))

245421

In [173]:
pangraph.H.number_of_nodes(), pangraph.H.number_of_edges()

(250091, 4876)

In [174]:
ref_data

'/data/hoan/amromics/data/ncbi/Ecolitest/GCF_014169015.1_ASM1416901v1_genomic.fna'

## Analysis

In [175]:
# minimap2 /data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna /data/hoan/amromics/genome-graph/scaffold_output/pangraph_opt_Ecoli100random_Ecoli_plasmid_v01.fasta > contigs100vsref_mapping_v01a.paf
cmd_minimap = '~/miniconda3/envs/amromics/bin/minimap2 '+ref_data +' '+ spades_output +'/contigs.fasta' +' > scaffold_output/minimap_out/mapping.paf' 

In [176]:
os.system(cmd_minimap)
df = pd.read_csv("scaffold_output/minimap_out/mapping.paf", sep ="\t", header=None)

[M::mm_idx_gen::0.204*1.01] collected minimizers
[M::mm_idx_gen::0.258*1.42] sorted minimizers
[M::main::0.258*1.42] loaded/built the index for 5 target sequence(s)
[M::mm_mapopt_update::0.292*1.37] mid_occ = 11
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 5
[M::mm_idx_stat::0.311*1.35] distinct minimizers: 938586 (97.76% are singletons); average occurrences: 1.036; average spacing: 5.349; total length: 5203192
[M::worker_pipeline::0.806*2.31] mapped 333 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: /home/vanhoan310/miniconda3/envs/amromics/bin/minimap2 /data/hoan/amromics/data/ncbi/Ecolitest/GCF_014169015.1_ASM1416901v1_genomic.fna /data/hoan/amromics/simulation/art_output/spades_output_Ecolitest_GCF_014169015_v01/contigs.fasta
[M::main] Real time: 0.821 sec; CPU: 1.876 sec; Peak RSS: 0.070 GB


In [177]:
df
ref_id = df.iloc[0,5]
ref_id

'NZ_AP022044.1'

In [178]:
final_df = df.loc[df[5] == ref_id]
final_df = final_df.sort_values(by=[7])

In [179]:
target_id = 0
gene_position = pd.read_csv(panta_output + '/gene_position.tsv', delimiter='\t', header=None)
gene_position.columns =['SampleID', 'ContigName', 'GeneSequence']
target_contigs = [gene_position.iloc[i, 1] for i in range(len(gene_position.index)) if gene_position.iloc[i, 0]==target_id]

In [180]:
target_contigs_list = [final_df.iloc[i, 0] for i in range(len(final_df.index)) if final_df.iloc[i, 0] in target_contigs]
target_contigs_sorted = [final_df.iloc[i, 0]+'---'+str(final_df.iloc[i, 7])+'>'+str(final_df.iloc[i, 8]) for i in range(len(final_df.index)) if final_df.iloc[i, 0] in target_contigs]

In [181]:
# gene_position[gene_position.ContigName=='NODE_90_length_903_cov_80.779481'].iloc[0,2].split(';')

In [182]:
# cluster_id = pangraph.gene2cluster_dict['0_5213@255@-1']

In [183]:
gene_info = pd.read_csv(panta_output + "/gene_info.tsv", delimiter='\t', header=None)
gene_info.columns =['GeneName', 'SampleID', 'clusterID']

In [184]:
# gene_info[gene_info.clusterID==cluster_id]

In [185]:
target_contigs_sorted[-4:]

['NODE_83_length_1298_cov_143.518906---4866903>4868194',
 'NODE_19_length_109099_cov_18.811397---4868771>4977867',
 'NODE_83_length_1298_cov_143.518906---4981400>4982691',
 'NODE_28_length_57572_cov_18.759167---4983278>5040837']

In [186]:
# pangraph.adj_list0

In [187]:
# cmd_dnadiff = '~/miniconda3/envs/amromics/bin/dnadiff '+ref_data +' '+ spades_output +'/contigs.fasta -p dna_diff/'+sim_name
# os.system(cmd_dnadiff)

In [188]:
# Don't use SPADES, shovil keep files.
# python readSimulator/readSimulator.py --outdir shredded_reads  --input ncbi-genomes-2023-01-07/GCF_000240185.1_ASM24018v2_genomic.fna --simulator art --simulator_path art_bin_MountRainier/art_illumina --iterations 100 --readlen 100 --depth 100  --opts '-ss HS20 -sam -p -m 400 -s 60'
# investigate why we failed? #1/ connectivity in the assembly graph:
# multiplicity of each node: now 1, change to more: use length and coverage
# assembly graph: change to 500

In [189]:
from pangraph.utils import get_node_coverage, get_node_length, get_value, next_node_multi_1, next_node_length

In [190]:
get_node_coverage('NODE_45_length_30843_cov_18.832500')
target_contigs_multi_1 = [item for item in target_contigs_list if get_node_coverage(item) <= 25.0]

In [191]:
# target_contigs_multi_1

In [192]:
# assembly_graph = pangraph.assembly_graph
# for i in range(len(target_contigs_list)-1):
#     print('---------------------- i =', i, '---------------------------------------------')
#     node1 = target_contigs_list[i]
#     node2 = target_contigs_list[i+1]
#     print(target_contigs_sorted[i],'::::::::', target_contigs_sorted[i+1])
#     source_id = 'C-' + str(pangraph.tail_contig[node1])
#     target_id = 'C-' + str(pangraph.head_contig[node2]) 
#     source_node = node1 + pangraph.strand[node1]
#     target_node = node2 + pangraph.strand[node2]
#     if assembly_graph.has_node(source_node) and assembly_graph.has_node(target_node):
#         print("Is there a path in the assembly graph? ", end ='')
#         print(nx.has_path(assembly_graph, source_node, target_node), end = ',')   
#         if nx.has_path(assembly_graph, source_node, target_node):
#             path_nucleotides = -1
#             path = nx.shortest_path(assembly_graph, source=source_node, target=target_node)
#             for idxp in range(1, len(path)-1):
#                 path_nucleotides = path_nucleotides + int(path[idxp].split("_")[3])
#             print(' number of nucleotides: ', path_nucleotides)
#         else:
#             print('<----xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
#     print("Is there a path in the pangenome graph? ", end ='')
#     if nx.has_path(pangraph.H, source_id, target_id):
#         p = nx.shortest_path(pangraph.H, source=source_id, target=target_id)
#         weight_p = 0.0
#         for node_p_idx in range(len(p)-1):
#             weight_p += pangraph.H[p[node_p_idx]][p[node_p_idx+1]]['weight']
#         if len(p) >= 2:
#             weight_p = 0.05 + weight_p/float((len(p)-1)*(len(p)-1))
#         print("Yes!, cost = ", weight_p)
#     else:
#         print("No path in Pangraph")
#     print("Real cost between the two contigs: ", end ='')
#     get_value(pangraph.edge_df0, node1, node2)   
#     # if get_node_coverage(node2) >= 25.0:
#     # node2 = next_node_multi_1(target_contigs_list, i+1)
#     if get_node_length(node1) >= 3000 and get_node_length(node2) <= 3000:
#         node2 = next_node_length(target_contigs_list, i+1, 5000)
#         print('******************************************************************************************************************************************************')
#         print(node1, node2)
#         source_id = 'C-' + str(pangraph.tail_contig[node1])
#         target_id = 'C-' + str(pangraph.head_contig[node2]) 
#         source_node = node1 + pangraph.strand[node1]
#         target_node = node2 + pangraph.strand[node2]
#         if assembly_graph.has_node(source_node) and assembly_graph.has_node(target_node):
#             print("Is there a path in the assembly graph? ", end ='')
#             print(nx.has_path(assembly_graph, source_node, target_node), end = ',')   
#             if nx.has_path(assembly_graph, source_node, target_node):
#                 path_nucleotides = -1
#                 path = nx.shortest_path(assembly_graph, source=source_node, target=target_node)
#                 for idxp in range(1, len(path)-1):
#                     path_nucleotides = path_nucleotides + int(path[idxp].split("_")[3])
#                 print(' number of nucleotides: ', path_nucleotides)
#             else:
#                 print('<----xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
#         print("Is there a path in the pangenome graph? ", end ='')
#         if nx.has_path(pangraph.H, source_id, target_id):
#             p = nx.shortest_path(pangraph.H, source=source_id, target=target_id)
#             weight_p = 0.0
#             for node_p_idx in range(len(p)-1):
#                 weight_p += pangraph.H[p[node_p_idx]][p[node_p_idx+1]]['weight']
#             if len(p) >= 2:
#                 weight_p = 0.05 + weight_p/float((len(p)-1)*(len(p)-1))
#             print("Yes!, cost = ", weight_p)
#         else:
#             print("No path in Pangraph")
#         print("Real cost between the two contigs: ", end ='')
#         get_value(pangraph.edge_df0, node1, node2)

In [193]:
# pangraph.edge_df0

In [194]:
# pangraph.edge_df0['source'].values

In [195]:
## Xet chieu trong trinh dai dien de sua roi inversion.

In [196]:
# Ideas:
# Nhiều cái nối bị sai vì các repetitive. (Xét long range relationship, hiện tại ta chỉ xét short range). 
# Cần phải có cái nhìn dài hơn. Giả sử là dùng greedy thì nếu thằng được xét có độ dài lớn, ta tính khoảng cách của nó đến một thằng
# có độ dài lớn, nếu khả năng nối là thấp thì ta bỏ qua và ko cho vào kết quả. 
# Quast circular???

In [197]:
import glob
import os
# list_file = glob.glob("/data/hoan/amromics/genome-graph/scaffold_output/quastResults_Kp50random_Kptest*/report.pdf")
# list_file = glob.glob("/data/hoan/amromics/genome-graph/scaffold_output/*Multiplicity_v01/report.pdf")
list_file = glob.glob("/data/hoan/amromics/genome-graph/scaffold_output/*0.4_v01/report.pdf")

In [198]:
# for file in list_file:
#     sample_id = file.split("/")[-2].split("_")[4]
#     cmd = 'mv ' + file +' '+ file[:-10] + str(sample_id) + '_t4.pdf'
#     print(cmd)
#     os.system(cmd)

In [199]:
# for i in range(len(target_contigs_sorted)):
#     print(target_contigs_sorted[i], end = ', ')
#     if (i+1)%3==0:
#         print('')

In [200]:
# pangraph.adj_list_assembly

In [201]:
# pangraph.adj_list0

In [202]:
# node1 = 'NODE_13_length_123136_cov_19.311462'
# node2 = 'NODE_4_length_210493_cov_18.979267'
# source_id = 'C-' + str(pangraph.tail_contig[node1])
# target_id = 'C-' + str(pangraph.head_contig[node2]) 
# source_node = node1 + pangraph.strand[node1]
# target_node = node2 + pangraph.strand[node2]

# if assembly_graph.has_node(source_node) and assembly_graph.has_node(target_node):
#     print("Is there a path in the assembly graph? ", end ='')
#     print(nx.has_path(assembly_graph, source_node, target_node), end = ',')   
#     if nx.has_path(assembly_graph, source_node, target_node):
#         path_nucleotides = -1
#         path = nx.shortest_path(assembly_graph, source=source_node, target=target_node)
#         for idxp in range(1, len(path)-1):
#             path_nucleotides = path_nucleotides + int(path[idxp].split("_")[3])
#         print(' number of nucleotides: ', path_nucleotides)
#     else:
#         print('<----xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
# print("Is there a path in the pangenome graph? ", end ='')
# if nx.has_path(pangraph.H, source_id, target_id):
#     p = nx.shortest_path(pangraph.H, source=source_id, target=target_id)
#     weight_p = 0.0
#     for node_p_idx in range(len(p)-1):
#         weight_p += pangraph.H[p[node_p_idx]][p[node_p_idx+1]]['weight']
#     if len(p) >= 2:
#         weight_p = 0.05 + weight_p/float((len(p)-1)*(len(p)-1))
#     print("Yes!, cost = ", weight_p)
# else:
#     print("No path in Pangraph")
# print("Real cost between the two contigs: ", end ='')
# get_value(pangraph.edge_df0, node1, node2)

In [203]:
# gene_position = pd.read_csv(panta_output + "/gene_position.tsv", delimiter='\t', header=None)
# gene_position.columns =['SampleID', 'ContigName', 'GeneSequence']
# gene_position_sub = gene_position[gene_position.SampleID==0]
# nodes_list = list(gene_position_sub.iloc[:,1].values)
# nodes_len = [int(node.split("_")[3]) for node in nodes_list]
# nodes_coverage = [float(node.split("_")[5]) for node in nodes_list]
# gene_position_sub['length'] = nodes_len
# gene_position_sub['coverage'] = nodes_coverage
# gene_position_sub = gene_position_sub.sort_values(by='length', ascending=False)
# basecoverage = np.median(gene_position_sub['coverage'][:5])
# expected_node_coverage = [min(9, round(gene_position_sub.iloc[i, 4]/basecoverage)) for i in range(len(nodes_coverage))]
# node_multiplicity = {}
# for i in range(len(nodes_coverage)):
#     node_multiplicity[gene_position_sub.iloc[i,1]] = expected_node_coverage[i]

In [204]:
# node_multiplicity

In [205]:
# nx.draw(pangraph.contig_graph)

In [206]:
# pangraph.contig_graph.nodes

In [207]:
# nx.write_gml(pangraph.contig_graph,'contig_gr.gml')